In [1]:
import json
import sys
import pandas as pd

sys.path.append('../')

from functions import *

/Users/liumukun/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data = pd.read_csv("../../data/demeaned.csv")
df = data.groupby(["id", "ispolice", "selleritemssold", "bidcount", "apple", "amazon", "increment_residual"])["residual"].apply(lambda x: x.values).reset_index()

valid_bids = list(df[df.ispolice == 1].bidcount.value_counts().index)
include = df[(df.bidcount > 1) & (df.bidcount.isin(valid_bids))]

bids = list(include.residual)

logged_feedback = np.log(include.selleritemssold+1)
logged_feedback = transform_covariates(logged_feedback, 100)
include.sellerfeedbackscore = logged_feedback

<ipython-input-2-eddbbdfb4a30>:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  include.sellerfeedbackscore = logged_feedback


### Import dumps

In [3]:
with open(f"./final dumps/CMF_alt1.json", "r") as f:
    d = json.load(f)
    covariates = d["covariates"]
    median_lower = d["lower"]
    median_upper = d["upper"]

#### Main specification

Table 12 Interval Estimate

In [20]:
def loss_function1(c):
    cef = lambda cov: c[0]+c[1]*cov[0]+c[2]*cov[1]*(1-cov[0])+c[3]*cov[1]*cov[0]
    return get_loss_function(covariates, median_upper, median_lower, cef)

point1, interval1 = get_estimates(loss_function1, 4, [0,0,0,0])

print(point1)
print(interval1)

[-0.63232709  0.43081826  0.08363734  0.00641672]
[-0.6455179024116361, -0.6192404277156456, 0.26171853426979436, 0.5437280649065365, 0.08176226350693923, 0.08548226978787891, -0.009530714656715344, 0.016982064701563965]


### Bootstrap Confidence Intervals

In [6]:
prefixed = [filename for filename in os.listdir("./final dumps") if filename.startswith("bootstrap_alt1")]

results = []

for name in prefixed:
    with open(f"./final dumps/{name}", "r") as f:
        d = json.load(f)
        covariates = d["covariates"]
        median_lower = d["lower"]
        median_upper = d["upper"]
        results.append((covariates, median_lower, median_upper))

#### Main specification

In [23]:
estimates1 = []

for i, result in enumerate(results):
    
    def loss_function1(c):
        cef = lambda cov: c[0]+c[1]*cov[0]+c[2]*cov[1]*(1-cov[0])+c[3]*cov[1]*cov[0]
        return get_loss_function(result[0], result[2], result[1], cef)
    
    point1, interval1 = get_estimates(loss_function1, 4, [0,0,0,0])
    
    estimates1.append(interval1)

Table 12 Confidence Interval

In [24]:
conf_interval = report_intervals(estimates1, 95)

95% confidence interval for variable 0:
[-0.8502268430563046, -0.27813032837357315]
95% confidence interval for variable 1:
[-0.11609824572417507, 1.7192659959612464]
95% confidence interval for variable 2:
[0.014670102595820399, 0.10219894110312767]
95% confidence interval for variable 3:
[-0.13060100530206373, 0.05486553131932128]
